In [1]:
!pip install -q langchain langchain-community google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [2]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
import os
from google.colab import userdata

# Access the API key from Colab's Secrets Manager
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [8]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [9]:
# tool create

@tool
def multiply(a:int, b:int) -> int:
  """given two numbers, returns the product """
  return a * b

In [10]:
multiply.invoke({"a":3,"b":4})

12

In [13]:
multiply.name

'multiply'

In [12]:
multiply.description

'given two numbers, returns the product'

In [14]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# tool binding

In [16]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [17]:
llm_with_tools = llm.bind_tools([multiply])
# not all the tools have tool binding functionality

In [ ]:
llm

**Tool calling : user calls the (custom) tools when necessary not the LLMs**

In [ ]:
llm_with_tools.invoke("hi how are you?")

In [ ]:
query = HumanMessage("multiply 10 with 30?")

In [ ]:
messages = [query]

In [ ]:
messages

In [ ]:
result = llm_with_tools.invoke(messages)

In [ ]:
messages.append(result)

In [ ]:
messages

# tool execution

In [ ]:
multiply.invoke(result.tool_calls[0])

In [ ]:
multiply.invoke(result.tool_calls[0]["args"])

In [ ]:
tool_result = multiply.invoke(result.tool_calls[0])

In [ ]:
messages.append(tool_result)

In [ ]:
llm_with_tools.invoke(messages)

# currency converter

# 🔗 https://app.exchangerate-api.com/dashboard/

In [31]:
os.environ["EXCHANGE_API_KEY"] = userdata.get('EXCHANGE_API_KEY')

In [33]:
EXCHANGE_API_KEY=(os.environ["EXCHANGE_API_KEY"])

In [42]:
# tool create

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """given a base currency and target currency by fetching, returns the conversion factor"""


  url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_API_KEY}/pair/{base_currency}/{target_currency}"

  response = requests.get(url)
  return response.json()


@tool
def convert(base_currency_value:int , conversion_rate:Annotated[float, InjectedToolArg]) -> float:
  """given a base currency value and conversion rate, returns the converted value"""
  return base_currency_value * conversion_rate

In [43]:
get_conversion_factor({'base_currency':'USD', 'target_currency':'NPR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1752364801,
 'time_last_update_utc': 'Sun, 13 Jul 2025 00:00:01 +0000',
 'time_next_update_unix': 1752451201,
 'time_next_update_utc': 'Mon, 14 Jul 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'NPR',
 'conversion_rate': 137.394}

In [44]:
convert.invoke({'base_currency_value':10, 'conversion_rate':137.394})

1373.94

In [ ]:
# tool binding
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [40]:
# tool calling
messages =[HumanMessage("find the conversion rate between USD and NPR ,and based on that convert 10 USD to NPR")]

In [41]:
messages

[HumanMessage(content='find the conversion rate between USD and NPR ,and based on that convert 10 USD to NPR', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # print(tool_calls)

  # execute the first tool and get the value of conversion rate

  if tool_call['name'] =="get_conversion_factor":
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # print(tool_message1)

    # fetch the conversion rate
    conversion_rate = (json.loads(tool_message1.content)['conversion_rate'])
    # print(tool_message1)



    # append the tool message to the message list
    messages.append(tool_message1)



  # execute the second tool and perform the conversion from tool-1

  if tool_call["name"] == 'convert':
    # fetch the correct arguement
    tool_call['arg']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [ ]:
llm_with_tools.invoke(messages).content